**Install packages**

In [1]:
!pip install pandas catboost scikit-learn xgboost lightgbm numpy==1.24.0 tqdm joblib shap optuna flask feature_engine networkx -q
# после установки - перезапустите окружение

In [1]:
from lazypredict.Supervised import LazyClassifier
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split, GroupKFold, GridSearchCV, ParameterGrid
from sklearn.metrics import roc_auc_score, precision_score, recall_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.impute import SimpleImputer
from feature_engine.imputation import MeanMedianImputer
import numpy as np
from tqdm import tqdm
import joblib
import shap
import warnings

warnings.filterwarnings('ignore')

In [16]:
pd.options.display.float_format = '{:.15f}'.format
models = pd.read_csv('models.csv')
models

,Model,Gini,RocAuc
0,RandomForestClassifier,0.601087329072902,NaN
1,CatBoostClassifier,0.593574896368103,NaN
2,XGBClassifier,0.587829107293975,NaN
3,LGBMClassifier,0.585739300129877,NaN
4,BaggingClassifier,0.563008177390122,NaN
5,SVM,0.436678197265473,NaN


In [17]:
models.RocAuc = models.Gini.apply(lambda x: (x + 1) / 2)

In [18]:
models

,Model,Gini,RocAuc
0,RandomForestClassifier,0.601087329072902,0.800543664536451
1,CatBoostClassifier,0.593574896368103,0.796787448184051
2,XGBClassifier,0.587829107293975,0.793914553646987
3,LGBMClassifier,0.585739300129877,0.792869650064939
4,BaggingClassifier,0.563008177390122,0.781504088695061
5,SVM,0.436678197265473,0.718339098632736


**Init data**

Merge X and y

In [ ]:
X = pd.read_csv('data/train_X.csv')
y = pd.read_csv('data/train_y.csv')
test = pd.read_csv('data/test2_X.csv')
train = X.merge(y, on=['contract_id', 'report_date'])
graph = pd.read_csv('data/graph.csv').drop(columns='Unnamed: 0').rename(columns={'contractor_id1': 'contractor_id'})
full_graph = graph.pivot_table(index='contractor_id', columns='contractor_id2', values='Distance').fillna(0).reset_index()
features_df = pd.read_excel('описание.xlsx')

In [3]:
"""
Делаем аггрегации над дистанциями в графе и группируем по contractor_id,
далее будем использовать эти фичи.
"""

connection_count = graph.groupby('contractor_id').size().to_dict()
mean_distance = graph.groupby('contractor_id').Distance.mean().to_dict()
min_distance = graph.groupby('contractor_id').Distance.min().to_dict()
max_distance = graph.groupby('contractor_id').Distance.max().to_dict()
var_distance = graph.groupby('contractor_id').Distance.var().to_dict()
median_distance = graph.groupby('contractor_id').Distance.median().to_dict()
std_distance = graph.groupby('contractor_id').Distance.std().to_dict()
count_distance = graph.groupby('contractor_id').Distance.count().to_dict()

**Feature selection**

In [4]:
class FeatureSelectionModel:
    """
    Class for the selection feature. 4 different models and 6 filters are used to select features.
    First, all models are trained to further extract their feature importance.
    Next, we extract shap-values ​​from the CatBoostClassifier model and the correlation with the target.
    After this, the top 280 features in each filter are searched and the intersections of the lists are selected.
    In this way, an average of 70-80 features out of 2100 are selected.
    """
    def __init__(self):
        self.cb_model = CatBoostClassifier(use_best_model=True, eval_metric='AUC')
        self.rf_model = RandomForestClassifier(5000, n_jobs=-1)
        self.xgb_model = XGBClassifier(n_estimators=100)
        self.lgb_model = LGBMClassifier(n_estimators=100, eval_metric='AUC')

    def fit(self, X, y):
        """Fit Classification algorithms to X and y.
        Parameters
        ----------
        X : array-like,
            Training vectors, where rows is the number of samples
            and columns is the number of features.
        y : array-like,
            Training vectors, where rows is the number of samples
            and columns is the number of features.
        """
        self.X = X
        self.y = y.map(int)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.1,
                                                                                random_state=42, stratify=self.y,
                                                                                shuffle=True)
        self.cb_model.fit(self.X_train, self.y_train, verbose=False, eval_set=(self.X_test, self.y_test))
        self.rf_model.fit(self.X_train, self.y_train)
        self.xgb_model.fit(self.X, self.y)
        self.lgb_model.fit(self.X, self.y)

    def get_feature_importance(self):
        """Get feature importance algorithms to data."""
        self.t = self.cb_model.get_feature_importance(prettified=True)

        self.rf = pd.DataFrame(self.rf_model.feature_importances_.T,
                               self.X_train.columns,
                               columns=['coef']) \
            .sort_values(by='coef', ascending=False)

        self.xgb = pd.DataFrame(list(self.xgb_model.feature_importances_),
                                self.X_train.columns,
                                columns=['coef']) \
            .sort_values(by='coef', ascending=False)

        self.lgb = pd.DataFrame(list(self.lgb_model.feature_importances_),
                                self.X_train.columns,
                                columns=['coef']) \
            .sort_values(by='coef', ascending=False)

        self.explainer = shap.TreeExplainer(self.cb_model)

        self.val_dataset = Pool(data=self.X_test, label=self.y_test)
        self.shap_values = self.explainer.shap_values(self.val_dataset)
        self.mean_abs_shap = np.abs(self.shap_values).mean(axis=0)
        self.fi_df = pd.DataFrame({
            'feature': self.X_test.columns,
            'importance': self.mean_abs_shap
        })
        self.fi_df = self.fi_df.sort_values(by='importance', ascending=False)

        self.temp = self.X.copy()
        self.temp['default6'] = self.y
        self.correlation = self.temp.corrwith(self.temp.default6).abs()

    def get_features(self) -> list:
        """Get features, which were selected.
        Returns
        -------
        features : list
            Returns features which were selected in list type.
        """
        self.get_feature_importance()
        self.cb_features = list(self.t[self.t.Importances > 0.01]['Feature Id'])[:280]
        self.shap_values_features = list(self.fi_df.head(281).feature)
        self.rf_features = list(self.rf.index)[:280]
        self.xgb_features = list(self.xgb.index)[:280]
        self.lgb_features = list(self.lgb.index)[:280]
        self.corr_features = list(
            self.correlation.sort_values(ascending=False).head(281).drop(columns='default6').index)
        self.total = []

        for i in self.corr_features:
            if i in self.cb_features and i in self.shap_values_features and i in self.rf_features and i in self.lgb_features and i in self.xgb_features:
                self.total.append(i)

        return self.total

**Feature generation**

В этом блоке кода мы создаем функции даты и финансовые атрибуты. На ставку по умолчанию влияет сезонность,
поэтому мы создадим функцию сезона, используя функцию get_ Season. Далее мы переведем функции Contract_date и report_date.
в формат даты и времени и извлекайте из них такие данные, как месяц, день, день недели и год. 
Также с помощью этих возможностей можно сделать функцию Contract_duration — длительность контракта путем вычитания одной даты из другой. 
Теперь вы можете обратить внимание на таблички с налогами и сделать новую функцию - сумму всех налогов. 
Также можно округлить атрибут с кредитами до 2 знаков после запятой, чтобы получилось что-то похожее на нормализованный класс — кредиты.
Нелишним будет сделать пару финансовых особенностей: превышают ли доходы расходы с налогами и без них.

In [5]:
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 0  # Зима
    elif month in [3, 4, 5]:
        return 1  # Весна
    elif month in [6, 7, 8]:
        return 2  # Лето
    else:
        return 3  # Осень


Функция создания дополнительных функций на основе дат, текущий и начальный контракт контракта, а также другие финансовые атрибуты.

In [6]:
def base_features(data: pd.DataFrame) -> pd.DataFrame:
    data = data.copy()
    data['season'] = pd.to_datetime(data['contract_date']).apply(get_season)  # сезон подписания контракта
    data['month'] = pd.to_datetime(data['contract_date']).dt.month.apply(int)  # месяц подписания контракта
    data['day'] = pd.to_datetime(data['contract_date']).dt.day.apply(int)  # день подписания контракта
    data['day_of_week'] = pd.to_datetime(data['contract_date']).dt.dayofweek.apply(
        int)  # день недели подписания контракта
    data['year'] = pd.to_datetime(data['contract_date']).dt.year.apply(int)  # год подписания контракта
    data['report_month'] = pd.to_datetime(data['report_date']).dt.month.apply(int)  # месяц среза
    data['report_day'] = pd.to_datetime(data['report_date']).dt.day.apply(int)  # день среза
    data['contract_duration'] = (pd.to_datetime(data['report_date']) - pd.to_datetime(
        data['contract_date'])).dt.days  # длительность контракта
    data['time'] = pd.to_datetime(data['contract_date']).astype(int) / 10 ** 11  # datetime -> timestamp
    data['report_time'] = pd.to_datetime(data['report_date']).astype(int) / 10 ** 11  # datetime -> timestamp
    data['total_claims_last_12_months'] = data['agg_ArbitrationCases__g_contractor__DefendantSum__sum__12M'] + data[
        'agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12M']  # сумма всех сборов за 12 месяцев
    data['total_claims_last_24_months'] = data['agg_ArbitrationCases__g_contractor__DefendantSum__sum__12_24M'] + data[
        'agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12_24M']  # сумма всех сборов за 24 месяца
    data['Income > Expenses'] = data['agg_FinanceAndTaxesFTS__g_contractor__Income__last__ALL_TIME'] > data[
        'agg_FinanceAndTaxesFTS__g_contractor__Expenses__last__ALL_TIME']  # доходы > расходы?
    data['Income > Expenses'] = data['Income > Expenses'].map(int)
    data['Income > Taxes + Expenses'] = data['agg_FinanceAndTaxesFTS__g_contractor__Income__last__ALL_TIME'] > (
                data['agg_FinanceAndTaxesFTS__g_contractor__TaxesSum__last__ALL_TIME'] + data[
            'agg_FinanceAndTaxesFTS__g_contractor__Expenses__last__ALL_TIME'])  # доходы > расходы + налоги?
    data['Income > Taxes + Expenses'] = data['Income > Taxes + Expenses'].map(int)
    data['credits'] = data['agg_spark_extended_report__g_contractor__CreditLimitSum__last__ALL_TIME'].round(
        2)  # округляем признак с кредитами
    data['tax'] = data['agg_FinanceAndTaxesFTS__g_contractor__TaxArrearsSum__last__ALL_TIME'] + data[
        'agg_FinanceAndTaxesFTS__g_contractor__TaxPenaltiesSum__last__ALL_TIME'] + data[
                      'agg_FinanceAndTaxesFTS__g_contractor__TaxesSum__last__ALL_TIME']  # сумма всех налогов
    data['contract_current_sum_mean_3M'] = data.groupby('contract_id')['contract_current_sum'].rolling(window=3,
                                                                                                       min_periods=1).mean().reset_index(
        0, drop=True)  # средняя сумма за 3 месяца по contract_id
    data['contract_sum_change_ratio'] = data['contract_current_sum'] / data['contract_init_sum']
    data['mean_weekly_abs_price_change'] = data[
        [f'agg_all_contracts__g_contract__abs_change_price_last_ds__isMain__last__ALL_TIME',
         f'agg_all_contracts__g_contract__abs_change_price_last_ds__isMain__mean__ALL_TIME']].mean(axis=1)
    data['arbitration_cases_12M'] = data[f'agg_ArbitrationCases__g_contractor__DefendantSum__sum__12M'] + \
                                    data[f'agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12M']

    return data

Функция создания дополнительных признаков на основе вычитания одних признаков из других,
а именно показатели за разные периоды времени. Таким образом, мы создаем приращение функции индикатора.
Также для каждого показателя создается атрибут среднего значения и изменения по сравнению со средним и исходным показателем.

In [7]:
def sums_features(data: pd.DataFrame) -> pd.DataFrame:
    data = data.copy()

    data['c1'] = data['agg_cec_requests__g_contract__request_id__all__count__2W'] - data[
        'agg_cec_requests__g_contract__request_id__all__count__1W']
    data['c2'] = data['agg_cec_requests__g_contract__request_id__all__count__3W'] - data[
        'agg_cec_requests__g_contract__request_id__all__count__2W']
    data['c3'] = data['agg_cec_requests__g_contract__request_id__all__count__4W'] - data[
        'agg_cec_requests__g_contract__request_id__all__count__3W']
    data['c4'] = data['agg_cec_requests__g_contract__request_id__all__count__5W'] - data[
        'agg_cec_requests__g_contract__request_id__all__count__4W']
    data['c5'] = data['agg_cec_requests__g_contract__request_id__all__count__6W'] - data[
        'agg_cec_requests__g_contract__request_id__all__count__5W']
    data['c6'] = data['agg_cec_requests__g_contract__request_id__all__count__7W'] - data[
        'agg_cec_requests__g_contract__request_id__all__count__6W']
    data['c7'] = data['agg_cec_requests__g_contract__request_id__all__count__8W'] - data[
        'agg_cec_requests__g_contract__request_id__all__count__7W']
    data['c8'] = data['agg_cec_requests__g_contract__request_id__all__count__12W'] - data[
        'agg_cec_requests__g_contract__request_id__all__count__8W']
    data['c9'] = data['agg_cec_requests__g_contract__request_id__all__count__ALL_TIME'] - data[
        'agg_cec_requests__g_contract__request_id__all__count__12W']
    data['c_mean'] = (data['c1'] + data['c2'] + data['c3'] + data['c4'] + data['c5'] + data['c6'] + data['c7'] + data[
        'c8'] + data['c9']) / 9
    data['c_change'] = data['c_mean'] / data['c1']

    data['s1'] = data['agg_cec_requests__g_contract__total_sum_accepted__all__sum__2W'] - data[
        'agg_cec_requests__g_contract__total_sum_accepted__all__sum__1W']
    data['s2'] = data['agg_cec_requests__g_contract__total_sum_accepted__all__sum__3W'] - data[
        'agg_cec_requests__g_contract__total_sum_accepted__all__sum__2W']
    data['s3'] = data['agg_cec_requests__g_contract__total_sum_accepted__all__sum__4W'] - data[
        'agg_cec_requests__g_contract__total_sum_accepted__all__sum__3W']
    data['s4'] = data['agg_cec_requests__g_contract__total_sum_accepted__all__sum__5W'] - data[
        'agg_cec_requests__g_contract__total_sum_accepted__all__sum__4W']
    data['s5'] = data['agg_cec_requests__g_contract__total_sum_accepted__all__sum__6W'] - data[
        'agg_cec_requests__g_contract__total_sum_accepted__all__sum__5W']
    data['s6'] = data['agg_cec_requests__g_contract__total_sum_accepted__all__sum__7W'] - data[
        'agg_cec_requests__g_contract__total_sum_accepted__all__sum__6W']
    data['s7'] = data['agg_cec_requests__g_contract__total_sum_accepted__all__sum__8W'] - data[
        'agg_cec_requests__g_contract__total_sum_accepted__all__sum__7W']
    data['s8'] = data['agg_cec_requests__g_contract__total_sum_accepted__all__sum__12W'] - data[
        'agg_cec_requests__g_contract__total_sum_accepted__all__sum__8W']
    data['s9'] = data['agg_cec_requests__g_contract__total_sum_accepted__all__sum__ALL_TIME'] - data[
        'agg_cec_requests__g_contract__total_sum_accepted__all__sum__12W']
    data['s_mean'] = (data['s1'] + data['s2'] + data['s3'] + data['s4'] + data['s5'] + data['s6'] + data['s7'] + data[
        's8'] + data['s9']) / 9
    data['s_change'] = data['s_mean'] / data['s1']

    data['m1'] = data['agg_cec_requests__g_contract__time_btw_requests__all__mean__2M'] - data[
        'agg_cec_requests__g_contract__time_btw_requests__all__mean__1M']
    data['m2'] = data['agg_cec_requests__g_contract__time_btw_requests__all__mean__3M'] - data[
        'agg_cec_requests__g_contract__time_btw_requests__all__mean__2M']
    data['m3'] = data['agg_cec_requests__g_contract__time_btw_requests__all__mean__4M'] - data[
        'agg_cec_requests__g_contract__time_btw_requests__all__mean__3M']
    data['m4'] = data['agg_cec_requests__g_contract__time_btw_requests__all__mean__5M'] - data[
        'agg_cec_requests__g_contract__time_btw_requests__all__mean__4M']
    data['m5'] = data['agg_cec_requests__g_contract__time_btw_requests__all__mean__6M'] - data[
        'agg_cec_requests__g_contract__time_btw_requests__all__mean__5M']
    data['m6'] = data['agg_cec_requests__g_contract__time_btw_requests__all__mean__7M'] - data[
        'agg_cec_requests__g_contract__time_btw_requests__all__mean__6M']
    data['m7'] = data['agg_cec_requests__g_contract__time_btw_requests__all__mean__8M'] - data[
        'agg_cec_requests__g_contract__time_btw_requests__all__mean__7M']
    data['m8'] = data['agg_cec_requests__g_contract__time_btw_requests__all__mean__12M'] - data[
        'agg_cec_requests__g_contract__time_btw_requests__all__mean__8M']
    data['m9'] = data['agg_cec_requests__g_contract__time_btw_requests__all__mean__ALL_TIME'] - data[
        'agg_cec_requests__g_contract__time_btw_requests__all__mean__12M']
    data['m_mean'] = (data['m1'] + data['m2'] + data['m3'] + data['m4'] + data['m5'] + data['m6'] + data['m7'] + data[
        'm8'] + data['m9']) / 9
    data['m_change'] = data['m_mean'] / data['m1']

    data['d1'] = data['agg_payments__g_contract__sum__all__countDistinct__2W'] - data[
        'agg_payments__g_contract__sum__all__countDistinct__1W']
    data['d2'] = data['agg_payments__g_contract__sum__all__countDistinct__4W'] - data[
        'agg_payments__g_contract__sum__all__countDistinct__2W']
    data['d3'] = data['agg_payments__g_contract__sum__all__countDistinct__8W'] - data[
        'agg_payments__g_contract__sum__all__countDistinct__4W']
    data['d4'] = data['agg_payments__g_contract__sum__all__countDistinct__12W'] - data[
        'agg_payments__g_contract__sum__all__countDistinct__8W']
    data['d5'] = data['agg_payments__g_contract__sum__all__countDistinct__ALL_TIME'] - data[
        'agg_payments__g_contract__sum__all__countDistinct__12W']
    data['d_mean'] = (data['d1'] + data['d2'] + data['d3'] + data['d4'] + data['d5']) / 5
    data['d_change'] = data['d_mean'] / data['d1']

    data['a1'] = data['agg_payments__g_contract__sum__all__sum__2W'] - data[
        'agg_payments__g_contract__sum__all__sum__1W']
    data['a2'] = data['agg_payments__g_contract__sum__all__sum__4W'] - data[
        'agg_payments__g_contract__sum__all__sum__2W']
    data['a3'] = data['agg_payments__g_contract__sum__all__sum__8W'] - data[
        'agg_payments__g_contract__sum__all__sum__4W']
    data['a4'] = data['agg_payments__g_contract__sum__all__sum__12W'] - data[
        'agg_payments__g_contract__sum__all__sum__8W']
    data['a5'] = data['agg_payments__g_contract__sum__all__sum__ALL_TIME'] - data[
        'agg_payments__g_contract__sum__all__sum__12W']
    data['a_mean'] = (data['a1'] + data['a2'] + data['a3'] + data['a4'] + data['a5']) / 5
    data['a_change'] = data['a_mean'] / data['a1']

    data['ac1'] = data['agg_ks2__g_contract__id__all__count__2W'] - data['agg_ks2__g_contract__id__all__count__1W']
    data['ac2'] = data['agg_ks2__g_contract__id__all__count__4W'] - data['agg_ks2__g_contract__id__all__count__2W']
    data['ac3'] = data['agg_ks2__g_contract__id__all__count__8W'] - data['agg_ks2__g_contract__id__all__count__4W']
    data['ac4'] = data['agg_ks2__g_contract__id__all__count__12W'] - data['agg_ks2__g_contract__id__all__count__8W']
    data['ac5'] = data['agg_ks2__g_contract__id__all__count__ALL_TIME'] - data[
        'agg_ks2__g_contract__id__all__count__12W']
    data['ac_mean'] = (data['ac1'] + data['ac2'] + data['ac3'] + data['ac4'] + data['ac5']) / 5
    data['ac_change'] = data['ac_mean'] / data['ac1']

    data['as1'] = data['agg_ks2__g_contract__total_sum__all__sum__2W'] - data[
        'agg_ks2__g_contract__total_sum__all__sum__1W']
    data['as2'] = data['agg_ks2__g_contract__total_sum__all__sum__4W'] - data[
        'agg_ks2__g_contract__total_sum__all__sum__2W']
    data['as3'] = data['agg_ks2__g_contract__total_sum__all__sum__8W'] - data[
        'agg_ks2__g_contract__total_sum__all__sum__4W']
    data['as4'] = data['agg_ks2__g_contract__total_sum__all__sum__12W'] - data[
        'agg_ks2__g_contract__total_sum__all__sum__8W']
    data['as5'] = data['agg_ks2__g_contract__total_sum__all__sum__ALL_TIME'] - data[
        'agg_ks2__g_contract__total_sum__all__sum__12W']
    data['as_mean'] = (data['as1'] + data['as2'] + data['as3'] + data['as4'] + data['as5']) / 5
    data['as_change'] = data['as_mean'] / data['as1']

    data['w1'] = data['agg_spass_applications__g_contract__appl_count_week__mean__2W'] - data[
        'agg_spass_applications__g_contract__appl_count_week__mean__1W']
    data['w2'] = data['agg_spass_applications__g_contract__appl_count_week__mean__3W'] - data[
        'agg_spass_applications__g_contract__appl_count_week__mean__2W']
    data['w3'] = data['agg_spass_applications__g_contract__appl_count_week__mean__4W'] - data[
        'agg_spass_applications__g_contract__appl_count_week__mean__3W']
    data['w4'] = data['agg_spass_applications__g_contract__appl_count_week__mean__5W'] - data[
        'agg_spass_applications__g_contract__appl_count_week__mean__4W']
    data['w5'] = data['agg_spass_applications__g_contract__appl_count_week__mean__6W'] - data[
        'agg_spass_applications__g_contract__appl_count_week__mean__5W']
    data['w6'] = data['agg_spass_applications__g_contract__appl_count_week__mean__8W'] - data[
        'agg_spass_applications__g_contract__appl_count_week__mean__6W']
    data['w7'] = data['agg_spass_applications__g_contract__appl_count_week__mean__12W'] - data[
        'agg_spass_applications__g_contract__appl_count_week__mean__8W']
    data['w8'] = data['agg_spass_applications__g_contract__appl_count_week__mean__26W'] - data[
        'agg_spass_applications__g_contract__appl_count_week__mean__12W']
    data['w9'] = data['agg_spass_applications__g_contract__appl_count_week__mean__ALL_TIME'] - data[
        'agg_spass_applications__g_contract__appl_count_week__mean__26W']
    data['w_mean'] = (data['w1'] + data['w2'] + data['w3'] + data['w4'] + data['w5'] + data['w6'] + data['w7'] + data[
        'w8'] + data['w9']) / 9
    data['w_change'] = data['w_mean'] / data['w1']

    data['f1'] = data['agg_workers__g_contract__fact_workers__all__mean__2W'] - data[
        'agg_workers__g_contract__fact_workers__all__mean__1W']
    data['f2'] = data['agg_workers__g_contract__fact_workers__all__mean__3W'] - data[
        'agg_workers__g_contract__fact_workers__all__mean__2W']
    data['f3'] = data['agg_workers__g_contract__fact_workers__all__mean__4W'] - data[
        'agg_workers__g_contract__fact_workers__all__mean__3W']
    data['f4'] = data['agg_workers__g_contract__fact_workers__all__mean__5W'] - data[
        'agg_workers__g_contract__fact_workers__all__mean__4W']
    data['f5'] = data['agg_workers__g_contract__fact_workers__all__mean__6W'] - data[
        'agg_workers__g_contract__fact_workers__all__mean__5W']
    data['f6'] = data['agg_workers__g_contract__fact_workers__all__mean__8W'] - data[
        'agg_workers__g_contract__fact_workers__all__mean__6W']
    data['f7'] = data['agg_workers__g_contract__fact_workers__all__mean__12W'] - data[
        'agg_workers__g_contract__fact_workers__all__mean__8W']
    data['f8'] = data['agg_workers__g_contract__fact_workers__all__mean__26W'] - data[
        'agg_workers__g_contract__fact_workers__all__mean__12W']
    data['f9'] = data['agg_workers__g_contract__fact_workers__all__mean__ALL_TIME'] - data[
        'agg_workers__g_contract__fact_workers__all__mean__26W']
    data['f_mean'] = (data['f1'] + data['f2'] + data['f3'] + data['f4'] + data['f5'] + data['f6'] + data['f7'] + data[
        'f8'] + data['f9']) / 9
    data['f_change'] = data['f_mean'] / data['f1']

    data['o1'] = data['agg_materials__g_contract__order_id__countDistinct__2W'] - data[
        'agg_materials__g_contract__order_id__countDistinct__1W']
    data['o2'] = data['agg_materials__g_contract__order_id__countDistinct__4W'] - data[
        'agg_materials__g_contract__order_id__countDistinct__2W']
    data['o3'] = data['agg_materials__g_contract__order_id__countDistinct__8W'] - data[
        'agg_materials__g_contract__order_id__countDistinct__4W']
    data['o4'] = data['agg_materials__g_contract__order_id__countDistinct__12W'] - data[
        'agg_materials__g_contract__order_id__countDistinct__8W']
    data['o5'] = data['agg_materials__g_contract__order_id__countDistinct__ALL_TIME'] - data[
        'agg_materials__g_contract__order_id__countDistinct__12W']
    data['o_mean'] = (data['o1'] + data['o2'] + data['o3'] + data['o4'] + data['o5']) / 5
    data['o_change'] = data['o_mean'] / data['o1']

    data['i1'] = data['agg_sroomer__g_contractor__sroomer_id__count__6M'] - data[
        'agg_sroomer__g_contractor__sroomer_id__count__3M']
    data['i2'] = data['agg_sroomer__g_contractor__sroomer_id__count__12M'] - data[
        'agg_sroomer__g_contractor__sroomer_id__count__6M']
    data['i3'] = data['agg_sroomer__g_contractor__sroomer_id__count__ALL_TIME'] - data[
        'agg_sroomer__g_contractor__sroomer_id__count__12M']
    data['i_mean'] = (data['i1'] + data['i2'] + data['i3']) / 3
    data['i_change'] = data['i_mean'] / data['i1']

    data['ds1'] = data['agg_ArbitrationCases__g_contractor__DefendantSum__sum__12_24M'] - data[
        'agg_ArbitrationCases__g_contractor__DefendantSum__sum__12M']
    data['ds2'] = data['agg_ArbitrationCases__g_contractor__DefendantSum__sum__12_36M'] - data[
        'agg_ArbitrationCases__g_contractor__DefendantSum__sum__12_24M']
    data['ds3'] = data['agg_ArbitrationCases__g_contractor__DefendantSum__sum__12_48M'] - data[
        'agg_ArbitrationCases__g_contractor__DefendantSum__sum__12_36M']
    data['ds4'] = data['agg_ArbitrationCases__g_contractor__DefendantSum__sum__ALL_TIME'] - data[
        'agg_ArbitrationCases__g_contractor__DefendantSum__sum__12_48M']
    data['ds_mean'] = (data['ds1'] + data['ds2'] + data['ds3'] + data['ds4']) / 4
    data['ds_change'] = data['ds_mean'] / data['ds1']

    data['p1'] = data['agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12_24M'] - data[
        'agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12M']
    data['p2'] = data['agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12_36M'] - data[
        'agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12_24M']
    data['p3'] = data['agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12_48M'] - data[
        'agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12_36M']
    data['p4'] = data['agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__ALL_TIME'] - data[
        'agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12_48M']
    data['p_mean'] = (data['p1'] + data['p2'] + data['p3'] + data['p4']) / 4
    data['p_change'] = data['p_mean'] / data['p1']

    data['cd1'] = data['agg_tender_proposal__g_contractor__id__ALL__countDistinct__2W'] - data[
        'agg_tender_proposal__g_contractor__id__ALL__countDistinct__1W']
    data['cd2'] = data['agg_tender_proposal__g_contractor__id__ALL__countDistinct__4W'] - data[
        'agg_tender_proposal__g_contractor__id__ALL__countDistinct__2W']
    data['cd3'] = data['agg_tender_proposal__g_contractor__id__ALL__countDistinct__8W'] - data[
        'agg_tender_proposal__g_contractor__id__ALL__countDistinct__4W']
    data['cd4'] = data['agg_tender_proposal__g_contractor__id__ALL__countDistinct__12W'] - data[
        'agg_tender_proposal__g_contractor__id__ALL__countDistinct__8W']
    data['cd5'] = data['agg_tender_proposal__g_contractor__id__ALL__countDistinct__26W'] - data[
        'agg_tender_proposal__g_contractor__id__ALL__countDistinct__12W']
    data['cd6'] = data['agg_tender_proposal__g_contractor__id__ALL__countDistinct__52W'] - data[
        'agg_tender_proposal__g_contractor__id__ALL__countDistinct__26W']
    data['cd7'] = data['agg_tender_proposal__g_contractor__id__ALL__countDistinct__ALL_TIME'] - data[
        'agg_tender_proposal__g_contractor__id__ALL__countDistinct__52W']
    data['cd_mean'] = (data['cd1'] + data['cd2'] + data['cd3'] + data['cd4'] + data['cd5'] + data['cd6'] + data[
        'cd7']) / 7
    data['cd_change'] = data['cd_mean'] / data['cd1']

    return data

**Simple model(without data preprocessing)**

Для начала, обучим обычную модель на дефолтных данных, без особой предобработки(только с заполнением пропусков),
удалим фичи, в которых много NaN и удалим дубликаты строчек, так как в 99% случаев показатель дефолта не меняется.
Об этом подробнее есть в ноутбуке EDA.ipynb, там наглядно видно, что подрядчик не меняет свой дефолт практически
в 100% случаев. Возьмем эту модель дальше для ансамбля.

In [9]:
train.isna().sum().sort_values()[::-1].head(10)

agg_FinanceAndTaxesFTS__g_contractor__TaxPenaltiesSum__last__ALL_TIME              25425
agg_all_contracts__g_contract__rel_change_price_last_ds__isMain__mean__ALL_TIME    23736
agg_all_contracts__g_contract__rel_change_price_last_ds__isMain__last__ALL_TIME    23736
agg_all_contracts__g_contract__abs_change_price_last_ds__isMain__mean__ALL_TIME    21507
agg_all_contracts__g_contract__abs_change_price_last_ds__isMain__last__ALL_TIME    21507
agg_FinanceAndTaxesFTS__g_contractor__TaxArrearsSum__last__ALL_TIME                18993
agg_sroomer__g_contractor__sroomer_id__count__ALL_TIME                             18377
agg_sroomer__g_contractor__sroomer_id__count__6M                                   18377
agg_sroomer__g_contractor__sroomer_id__count__3M                                   18377
agg_sroomer__g_contractor__sroomer_id__count__12M                                  18377
dtype: int64

Есть некоторые функции, которые содержат много NaN, разумно будет не использовать их и отказаться:
agg_all_contracts__g_contract__abs_change_price_last_ds__isMain__last__ALL_TIME,
agg_all_contracts__g_contract__abs_change_price_last_ds__isMain__mean__ALL_TIME,
agg_all_contracts__g_contract__rel_change_price_last_ds__isMain__last__ALL_TIME,
agg_all_contracts__g_contract__rel_change_price_last_ds__isMain__mean__ALL_TIME,
agg_FinanceAndTaxesFTS__g_contractor__TaxPenaltiesSum__last__ALL_TIME

In [11]:
to_drop = ['agg_all_contracts__g_contract__abs_change_price_last_ds__isMain__last__ALL_TIME',
            'agg_all_contracts__g_contract__abs_change_price_last_ds__isMain__mean__ALL_TIME',
            'agg_all_contracts__g_contract__rel_change_price_last_ds__isMain__last__ALL_TIME',
            'agg_all_contracts__g_contract__rel_change_price_last_ds__isMain__mean__ALL_TIME',
            'agg_FinanceAndTaxesFTS__g_contractor__TaxPenaltiesSum__last__ALL_TIME']

In [12]:
copy_train = train.copy()
copy_train['default6'] = y.default6

In [13]:
features = []
for i in copy_train.isnull().sum().items():
    if i[-1] > len(copy_train) * 0.7:
        to_drop.append(i[0])
    if 0 < i[-1] <= len(copy_train) * 0.7:       
        features.append(i[0])

In [14]:
"""Fill NaN by median"""
copy_train = copy_train.drop(columns=to_drop)
imputer = MeanMedianImputer(imputation_method='median', variables=features)
imputer.fit(copy_train[features])
copy_train[features] = imputer.transform(copy_train[features])

In [15]:
copy_train = copy_train.drop(columns=['report_date', 'contract_date'])
copy_train = copy_train.drop_duplicates(subset=['contract_id', 'contractor_id', 'default6'])
copy_train = copy_train.groupby(['contract_id', 'contractor_id', 'specialization_id', 'project_id'], as_index=False).mean()
copy_train = copy_train.drop(columns=['contract_id','project_id', 'building_id'])

In [16]:
copy_test1 = test.copy()
copy_test1 = copy_test1.drop(columns=['contract_id','project_id', 'building_id'])
copy_test1[features] = imputer.transform(copy_test1[features])

In [17]:
dd = {}
contractors2 = list(set(copy_train.contractor_id))

In [18]:
for index, row in graph.iterrows():
    if row.contractor_id not in dd:
        dd[row.contractor_id] = []
    if row.contractor_id2 in contractors2:
        dd[row.contractor_id].append((row.contractor_id2, row.Distance))

In [19]:
for j in dd:
    dd[j] = [i[0] for i in sorted(dd[j], key=lambda x: x[1])][:1]

In [20]:
X_simple = copy_train.drop(columns=['default6', 'contractor_id'])
y_simple = copy_train['default6'].map(int)

In [21]:
simple_model = RandomForestClassifier(class_weight='balanced', max_depth=20,
                       min_samples_leaf=2, n_estimators=500, random_state=42)
simple_model.fit(X_simple, y_simple);

In [22]:
joblib.dump(simple_model, 'final/simple_model.joblib')

['final/simple_model.joblib']

В итоге, у нас готова первая модель для ансамбля. Дальше попробуем сделать модели, обученные
на данных с хорошей предобработкой и генерацией дополнительных фичей. Так как это самая слабая модель,
разумнее всего будет ей поставить небольшой коэфициента для блендинга предсказаний.

**Data preprocessing for best models**

Применим функции к данным для генерации дополнительных признаков, так как доп. признаки - это
всегда хорошо, у модели будет больше вероятность дать правильный ответ.
Обрабатываем пропуски средним, делаем one-hot кодирование признаков с ID идентификаторами.
Таким образом после фича селекшена модель отберет несколько айдишников, по которым можно
довольно точно определить дефолт. У этого способа есть минус - долгое обучение модели.

In [25]:
train = base_features(train)
test = base_features(test)

In [26]:
train = sums_features(train)
test = sums_features(test)

Сохранение Contract_id для перекрестной проверки
Заполняем пропуски в среднем столбце. Затем преобразуем id обратно в int, так как SimpleImputer преобразует все в число с плавающей запятой.
Мы выполняем горячее кодирование для функций с идентификаторами, чтобы модель обращала больше внимания на определенные значения.

In [27]:
ctrs = train.contract_id
contractors1 = train.contractor_id
contractors2 = test.contractor_id

In [28]:
train = train.drop(columns=['report_date', 'contract_date', 'contract_id'])  # delete dates и contract_id
full_df = pd.concat(
    [train.drop(columns='default6'), test.drop(columns=['report_date', 'contract_date', 'contract_id'])],
    ignore_index=True) # concating train and test

In [29]:
# id -> int type
full_df['specialization_id'] = full_df['specialization_id'].map(int)
full_df['project_id'] = full_df['project_id'].map(int)
full_df['building_id'] = full_df['building_id'].map(int)
full_df['contractor_id'] = full_df['contractor_id'].map(int)

In [30]:
# one-hot encoding for ID features
specs = pd.get_dummies(full_df.specialization_id).map(int).rename(columns=dict(
    zip(full_df.specialization_id.unique().tolist(),
        [f'spec_{i}' for i in full_df.specialization_id.unique().tolist()])))
projects = pd.get_dummies(full_df.project_id).map(int).rename(
    columns=dict(zip(full_df.project_id.unique(), [f'project_{i}' for i in full_df.project_id.unique().tolist()])))
buildings = pd.get_dummies(full_df.building_id).map(int).rename(columns=dict(
    zip(full_df.building_id.unique().tolist(), [f'building_{i}' for i in full_df.building_id.unique().tolist()])))
contractors = pd.get_dummies(full_df.contractor_id).map(int).rename(columns=dict(
    zip(full_df.contractor_id.unique().tolist(), [f'contractor_{i}' for i in full_df.contractor_id.unique().tolist()])))

In [31]:
# join ids
full_df = full_df.join(specs)
full_df = full_df.join(projects)
full_df = full_df.join(buildings)
full_df = full_df.join(contractors)

In [32]:
full_df1 = full_df.copy()
full_df2 = full_df.copy()

**Data preprocessing for default6 percentage on R distance**

Далее проверим гипотезу - создание признака с процентом дефолта в радиусе R.
При помощи графа отберем для каждого подрядчика тех, что находятся в радиусе 20,
посчитаем общую сумму их дефолтов и поделим на количество дефолтов. Таким образом,
у нас появляется новый признак, который далее будет для модели самым значимым. Это голд-фича
нашего решения, ведь это дало нам самый большой бустинг скора на валидации и на лидерборде(примерно на 0.01).
Также добавим пару аггрегационных фич, которые сделали в самом начале ноутбука. Это дает неплохой прирост
для скора. Самыми лучшими из них будут минимальное расстояние до подрядчиков. Это помогает модели
понять, насколько далеко от остальных находится подрядчик. Например, если у него минимальное расстояние будет
довольно большое, логично предположить, что с ним плохая идет работа, так как он ни с кем не сотрудничает.
Чтобы понимать примерно какие расстояния между подрядчика, добавим признак медианное расстояние, а также его
дисперсия. Это также помогает понять модели, какие в целом будут расстояния. Если медиана небольшая, то
вероятнее всего этот подрядчик много с кем сотрудничает. А если дисперсия большая, то модель может понять,
что скорее всего в данных есть выбросы. Например, в среднем он сотрудничает с подрядчиками на расстоянии до 30,
а есть связь с несколькими подрядчиками на расстоянии 30000, что делает дисперсию довольно большой.
Точно также заполняем пропуски средним значением.

In [34]:
def default_percentage_radius(r: int) -> pd.DataFrame:
    """
    Функция для поиска процентов по умолчанию 6 в радиусе R.
    Для каждого подрядчика выбираются другие подрядчики на расстоянии до R включительно.
    Далее вычисляется процент default6 путем деления суммы всех default6 на их количество.
    """
    default6_proc = []

    for contractor_id in tqdm(list(full_df.contractor_id.unique())):
        l = list(graph[(graph.Distance <= r) & (graph.contractor_id == contractor_id)].contractor_id2.unique())
        cnt = 0
        default6 = 0

        for _, row in train.iterrows():
            if row.contractor_id in l or row.contractor_id == contractor_id:
                cnt += 1
                default6 += row.default6

        if cnt != 0:
            default6_proc.append({
                "contractor_id": contractor_id,
                "proc_default6": default6 / cnt
            })
        else:
            default6_proc.append({
                "contractor_id": contractor_id,
                "proc_default6": 0
            })

    return pd.DataFrame(default6_proc)

In [35]:
default6_proc = default_percentage_radius(20)

100%|██████████| 847/847 [07:55<00:00,  1.78it/s]


In [36]:
# агрегации над расстояниями

def distance_aggregations_features(df: pd.DataFrame, graph: pd.DataFrame) -> pd.DataFrame:
    """Функция для создания признаков агрегирования на основе расстояний на графике и данных df. Возвращает pd.DataFrame с функциями"""
    data = []

    for i in list(df.contractor_id.unique()):
        try:
            a = graph[(graph.contractor_id == i) | (graph.contractor_id2 == i)].sort_values(by='Distance').head(5)
            l = []
            for _, row in a.iterrows():
                if row.contractor_id == i:
                    l.append(row.contractor_id2)
                else:
                    l.append(row.contractor_id)

            data.append({
                "contractor_id": i,
                "min_distance": min_distance[i],
                "max_distance": max_distance[i],
                "var_distance": var_distance[i],
                "median_distance": median_distance[i],
            })
        except:
            data.append({
                "contractor_id": i,
                "min_distance": 0,
                "max_distance": 0,
                "var_distance": 0,
                "median_distance": 0,
        })

    return pd.DataFrame(data)

In [37]:
data = distance_aggregations_features(full_df, graph)

In [38]:
full_df = full_df.merge(data, on='contractor_id') # merge full_df and data

In [39]:
full_df = full_df.merge(default6_proc, on='contractor_id') # merge default6_proc and data

In [40]:
# fill NaN by mean values
imputer = SimpleImputer(strategy='mean')
df_imputed = imputer.fit_transform(full_df)
df_imputed = pd.DataFrame(df_imputed, columns=full_df.columns)

In [41]:
df_imputed = df_imputed.drop(columns=['specialization_id', 'project_id', 'building_id', 'contractor_id'])  # drop id

In [42]:
X = df_imputed[:len(train)]  # slice by train
y = train.default6

In [43]:
fsmodel = FeatureSelectionModel() # model for Feature Selection

In [44]:
fsmodel.fit(X, y)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 4704, number of negative: 24127
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017188 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 55300
[LightGBM] [Info] Number of data points in the train set: 28831, number of used features: 949
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.163158 -> initscore=-1.634918
[LightGBM] [Info] Start training from score -1.634918


In [45]:
total = fsmodel.get_features() # a list of features to fit final model

In [46]:
len(total) # quantity of selected features

72

In [47]:
fsmodel.fi_df

,feature,importance
2087,proc_default6,0.42
178,contract_current_sum_mean_3M,0.24
1,contract_current_sum,0.19
129,agg_spark_extended_report__g_contractor__Credi...,0.17
171,report_time,0.12
...,...,...
873,building_529,0.00
871,building_527,0.00
869,building_525,0.00
868,building_524,0.00


Как и ожидалось, этот признак является наиболее значимым для модели. Поэтому дальше можно попробовать поработать
с соседями подрядчика, так как выяснилось, что показатель дефолта его соседей сильно связаны с дефолтом самого подрядчика.
Скорее всего(наше предположение), что если подрядчик сотрудничает с хорошими людьми, выполняющими свою работу, то и
он сам ее выполнит, а если с плохими, невыполняющими свои обязательства, аналогично не выполнит заказ.

**Data Preprocessing top 5 neighbours**

Выдвинем гипотезу, что показатель дефолта подрядчика сильно зависит от показателя дефолта его ближайших соседей.
Как мы выяснили в прошлом блоке, это оказалось правдой. Можно попробовать провести некоторые
операции над соседями подрядчиков. По таблицам с feature importance, мы увидели, что наиболее значимыми
являются признаки contract_init_sum и contract_current_sum. Почему бы не отобрать 5 ближайших подрядчиков
и использовать их признаки. Но так как в 1 строчку нам нужно подставить значение сразу за все строчки подрядчика,
лучше всего будет усреднять эти показатели по контрактору и записывать их в датафрейм. Также, применим аггрегационные фичи,
которые делали в прошлом шаге. После обучения, включим эту модель в ансамбль.

In [50]:
def select_top5_neighbours(df: pd.DataFrame, graph: pd.DataFrame) -> pd.DataFrame:
    """Функция выбора топ-5 ближайших контрагентов. По каждому подрядчику
    5 ближайших из них выбираются и сохраняются в кадре данных.
    Если контрагентов нет, во все столбцы добавляются ноли.
    """
    data = []

    for i in list(df.contractor_id.unique()):
        try:
            a = graph[(graph.contractor_id == i) | (graph.contractor_id2 == i)].sort_values(by='Distance').head(5)
            l = []
            for _, row in a.iterrows():
                if row.contractor_id == i:
                    l.append(row.contractor_id2)
                else:
                    l.append(row.contractor_id)

            data.append({
                "contractor_id": i,
                "mean_distance": mean_distance[i],
                "min_distance": min_distance[i],
                "max_distance": max_distance[i],
                "var_distance": var_distance[i],
                "median_distance": median_distance[i],
                "top1": l[0],
                "top2": l[1],
                "top3": l[2],
                "top4": l[3],
                "top5": l[4],
            })
        except:
            data.append({
                "contractor_id": i,
                "mean_distance": 0,
                "min_distance": 0,
                "max_distance": 0,
                "var_distance": 0,
                "median_distance": 0,
                "top1": 0,
                "top2": 0,
                "top3": 0,
                "top4": 0,
                "top5": 0,
            })

    return pd.DataFrame(data)

In [51]:
data = select_top5_neighbours(full_df1, graph)

In [52]:
data

,contractor_id,mean_distance,min_distance,max_distance,var_distance,median_distance,top1,top2,top3,top4,top5
0,438,20.98,12,35,36.42,20.00,656,709,507,801,278
1,484,547.79,12,32767,16923735.90,24.00,607,321,604,727,532
2,500,635.33,12,32767,19852958.65,19.00,511,491,626,634,455
3,615,0.00,0,0,0.00,0.00,0,0,0,0,0
4,633,473.55,12,32767,14433868.40,26.00,709,683,270,605,243
...,...,...,...,...,...,...,...,...,...,...,...
842,18,520.94,12,32767,16183360.16,18.00,267,727,506,592,879
843,718,440.34,16,32767,13396562.02,29.00,813,738,771,43,200
844,631,739.94,12,32767,22996293.67,24.00,776,880,141,353,587
845,321,740.03,12,32767,22996137.32,25.00,545,484,231,524,132


In [53]:
full_df1 = full_df1.merge(data, on='contractor_id')

In [54]:
def top5_neighbours_features(df: pd.DataFrame, graph: pd.DataFrame) -> pd.DataFrame: 
    d = []

    for _, row in tqdm(df.iterrows()):
        try:
            top1 = df[df.contractor_id == row.top1]
            top2 = df[df.contractor_id == row.top2]
            top3 = df[df.contractor_id == row.top3]
            top4 = df[df.contractor_id == row.top4]
            top5 = df[df.contractor_id == row.top5]

            top1init = top1.contract_init_sum.mean()
            top1curr = top1.contract_current_sum.mean()

            top2init = top2.contract_init_sum.mean()
            top2curr = top2.contract_current_sum.mean()

            top3init = top3.contract_init_sum.mean()
            top3curr = top3.contract_current_sum.mean()

            top4init = top4.contract_init_sum.mean()
            top4curr = top4.contract_current_sum.mean()

            top5init = top5.contract_init_sum.mean()
            top5curr = top5.contract_current_sum.mean()

            d.append({
                "top1init": top1init,
                "top1curr": top1curr,
                "top2init": top2init,
                "top2curr": top2curr,
                "top3init": top3init,
                "top3curr": top3curr,
                "top4init": top4init,
                "top4curr": top4curr,
                "top5init": top5init,
                "top5curr": top5curr
            })
        except:
            d.append({
                "top1init": 0,
                "top1curr": 0,
                "top2init": 0,
                "top2curr": 0,
                "top3init": 0,
                "top3curr": 0,
                "top4init": 0,
                "top4curr": 0,
                "top5init": 0,
                "top5curr": 0
            })

    return pd.DataFrame(d)

In [55]:
data = top5_neighbours_features(full_df1, graph)

42047it [03:12, 218.48it/s]


In [56]:
full_df1 = pd.concat([full_df1, data], axis=1)

In [57]:
# fill NaN by mean
imputer1 = SimpleImputer(strategy='mean')
df_imputed1 = imputer.fit_transform(full_df1)
df_imputed1 = pd.DataFrame(df_imputed1, columns=full_df1.columns)

In [58]:
df_imputed1 = df_imputed1.drop(
    columns=['specialization_id', 'project_id', 'building_id', 'contractor_id'])  # drop id

In [59]:
df_imputed1 = df_imputed1.drop(columns=['top1', 'top2', 'top3', 'top4', 'top5'])  # drop top 5 contractor ids

In [60]:
X2 = df_imputed1[:len(train)]  # slice by train

In [61]:
fsmodel1 = FeatureSelectionModel()
fsmodel1.fit(X2, y)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 4704, number of negative: 24127
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022259 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 57627
[LightGBM] [Info] Number of data points in the train set: 28831, number of used features: 959
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.163158 -> initscore=-1.634918
[LightGBM] [Info] Start training from score -1.634918


In [62]:
total1 = fsmodel1.get_features()

In [63]:
len(total1)

68

In [64]:
fsmodel1.fi_df

,feature,importance
178,contract_current_sum_mean_3M,0.33
1,contract_current_sum,0.21
129,agg_spark_extended_report__g_contractor__Credi...,0.19
170,time,0.15
2093,top3curr,0.14
...,...,...
1007,building_669,0.00
22,agg_cec_requests__g_contract__total_sum_accept...,0.00
1005,building_667,0.00
1004,building_666,0.00


Собственно, что и ожидалось. Действительно, показатели соседей очень влияют на дефолт подрядчика.
Скорее всего, если у подрядчика большие суммы на его счету, то у него будут средства на материалы и расходы,
что увеличивает вероятность того, что он выполнит свой заказ. Видим, что в топе признаков по важности
есть фичи, которые мы только что сделали - top1init и top3curr.

**Baseline preprocessing**

Попробуем обучить модель без особой предобработки для того, чтобы распределение
скора отличалось от моделей с хорошей предобработкой. Так как при ансамблировании
этих моделей при разных распределениях, они исправляют ошибки друг друга, что сильно
бустит показатель скора. Так как модель с плохой предобработкой, поставим ей небольшой коэфициент.

In [67]:
# fill NaN by mean
imputer2 = SimpleImputer(strategy='mean')
df_imputed2 = imputer2.fit_transform(full_df2)
df_imputed2 = pd.DataFrame(df_imputed2, columns=full_df2.columns)

In [68]:
df_imputed2 = df_imputed2.drop(
    columns=['specialization_id', 'project_id', 'building_id', 'contractor_id'])  # удаляем id

In [69]:
X3 = df_imputed2[:len(train)]  # slice by train

In [70]:
fsmodel2 = FeatureSelectionModel()
fsmodel2.fit(X3, y)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 4704, number of negative: 24127
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024560 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 54717
[LightGBM] [Info] Number of data points in the train set: 28831, number of used features: 944
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.163158 -> initscore=-1.634918
[LightGBM] [Info] Start training from score -1.634918


In [71]:
total2 = fsmodel2.get_features()

In [72]:
len(total2)

73

**Time series Stemming**

В нашем решении мы использовали временные ряды и стемминг
Шаги реализации:
1. Делаем стемминг на всех данных для получения предсказаний, которые отображают вероятность дефолта определенной записи
у подрядчика, не делая глубокий анализ по нему, как бы отражая вероятность дефолта вообщем для таких данных. Значения,
полученные со стемминга помогают модели временного ряда давать более точные предсказания, так как без стемминга модель бы
давала менее точные предикты в связи с небольшим количеством записей у каждого подрядчика. Модели временного ряда являются
узконаправленными. В качестве моделей для стемминга мы использовали RandomForestClassifier и CatBoostClassifier, так как они
лучше всего подходят для данного набора данных(В этом ноутбуке есть доказательство этого с помощью библиотеки LazyPredict).
Записываем вероятности, полученные со стемминга в коллонку 'score_from_cross_val'.
2. Группируем по train датасет contractor_id
3. Обучаем свою модель для каждой группы строк, в которой одинаковый contractor_id, имеющий больше 20 записей в
трейне(Значение в 20 записей было выбрано для того чтобы избежать неточных результатов от моделей, обученных на менее чем 20 записей)
4. Для того чтобы получить предсказания с теста, мы записываем предикты основной модели и колонку 'score_from_cross_val' и предсказываем вероятности дефолта на тестовых данных
5. Записываем полученные предикты группы моделей с весом 0,33 и предикты от основной модели с весом 0.67


In [74]:
class TimeSeries:
    def __init__(self):
        """A class for a Time Series Stemming model that will make model-based predictions,
        which were trained on data over different periods of time. The class implements the following methods:
        fit, predict_proba, submit, init_models, time_series_predict.
        """
        self.rf_model1 = RandomForestClassifier(5000, n_jobs=-1)
        self.rf_model2 = RandomForestClassifier(5000, n_jobs=-1)
        self.cb_model1 = CatBoostClassifier(2000, verbose=False)
        self.cb_model2 = CatBoostClassifier(2000, verbose=False)

    def fit(self, X, y):
        """Fit Classification algorithms to X_train and y_train.
        Parameters
        ----------
        X : array-like,
            Training vectors, where rows is the number of samples
            and columns is the number of features.
        y : array-like,
            Training vectors, where rows is the number of samples
            and columns is the number of features.
        """
        self.X = X.copy()
        self.y = y.map(int)

        self.X['score_from_cross_val'] = [0] * len(X)
        self.X1, self.X2, self.y1, self.y2 = train_test_split(self.X, self.y, test_size=0.5, shuffle=False)

        self.rf_model1.fit(self.X1, self.y1)
        self.rf_model2.fit(self.X2, self.y2)

        self.cb_model1.fit(self.X1, self.y1)
        self.cb_model2.fit(self.X2, self.y2)

    def predict_proba(self):
        """Prediction probability algorithms to data."""
        self.p1 = self.rf_model2.predict_proba(self.X1)[:, 1]
        self.p2 = self.cb_model2.predict_proba(self.X1)[:, 1]
        self.preds1 = self.p1 * 0.8 + self.p2 * 0.2

        self.X1['score_from_cross_val'] = self.preds1

        self.p3 = self.rf_model1.predict_proba(self.X2)[:, 1]
        self.p4 = self.cb_model1.predict_proba(self.X2)[:, 1]
        self.preds2 = self.p3 * 0.8 + self.p4 * 0.2

        self.X2['score_from_cross_val'] = self.preds2

        return pd.concat([self.X1, self.X2])

    def submit(self, X):
        """Prediction probability algorithms to data.
        Parameters
        ----------
        X : array-like,
            Testing vectors, where rows is the number of samples
            and columns is the number of features.
        """
        X1, X2, y1, y2 = train_test_split(X, y, test_size=0.5, shuffle=False)

        p1 = self.rf_model2.predict_proba(X1)[:, 1]
        p2 = self.cb_model2.predict_proba(X1)[:, 1]
        preds1 = p1 * 0.8 + p2 * 0.2

        X1['score_from_cross_val'] = preds1

        p3 = self.rf_model1.predict_proba(X2)[:, 1]
        p4 = self.cb_model1.predict_proba(X2)[:, 1]
        preds2 = p3 * 0.8 + p4 * 0.2

        X2['score_from_cross_val'] = preds2

        return pd.concat([X1, X2])

    def init_models(self, data):
        """Fit Classification algorithms to data.
        Parameters
        ----------
        data : array-like,
            Testing vectors, where rows is the number of samples
            and columns is the number of features.
        """
        total3 = total + ['contractor_id', 'score_from_cross_val', 'score']
        for_each = data[total3].groupby('contractor_id')
        self.models = {}
        for name, group in tqdm(for_each):
            if group.shape[0] > 20:
                mod = CatBoostClassifier(allow_const_label=True, verbose=False)
                mod.fit(group.drop(columns=['score']), group['score'])
                self.models[name] = mod
        print("DONE")

    def time_series_predict(self, data):
        """Prediction probability algorithms to data.
        Parameters
        ----------
        X : array-like,
            Testing vectors, where rows is the number of samples
            and columns is the number of features.
        """
        tt = data.groupby('contractor_id')
        fn = pd.DataFrame(columns=["c", 'score2'])
        count = 0
        total_ts = total + ['contractor_id', 'score_from_cross_val']
        for name, group in tqdm(tt):
            if name in self.models:
                ids = group['c']
                predd = self.models[name].predict_proba(group[total_ts])[:, 1]
                mem = pd.DataFrame({"c": ids, 'score2': predd})
                fn = pd.concat([fn, mem])
                count += 1

        submit2 = data.merge(fn, on='c', how="left")
        submit2['score'][submit2['score2'].notnull()] = (2 * submit2['score'] + submit2['score2']) / 3
        # print(submit2.score2)
        return (submit2.drop(columns=['score2', 'c']), submit2)

    def save_model(self, path: str):
        """Model save algorithms.
        Parameters
        ----------
        path : str,
            String with path you need to save model.
        """
        joblib.dump(self.models, path)

    @classmethod
    def load_model(cls, path: str):
        """Load the models from a file and return an instance of TimeSeries."""
        instance = cls()
        instance.models = joblib.load(path)
        print(f"Models loaded from {path}")
        return instance

In [75]:
ts = TimeSeries()
ts.fit(X, y)

In [76]:
X1 = ts.predict_proba() # dataframe with stemming on score_from_cross_val

In [77]:
X1['contractor_id'] = contractors1

In [78]:
f = X1.copy()
f['score'] = y

In [79]:
f['contractor_id'] = contractors1

In [80]:
ts.init_models(f) # fitting models

100%|██████████| 646/646 [04:37<00:00,  2.33it/s]

DONE


**Check contract for valid**

Выдвинем гипотезу, что если контрак плохой и его никто не выполняет, то
соотвественно, вероятность его выполнения у любого подрядчика будет довольно низка.
Проверим это следующим способом. Для каждого контракта будем брать рандомного подрядчика
и подставлять ему заместо признаков, связанных с контрактом, признаки контракта, который хотим проверить.
Если подрядчик не сможет выполнить эту работу, то скорее всего контракт действительно плохой.
Далее сблендим предсказания в соотношении 1:1.
Данный алгоритм улучшает предсказания нашей основной модели. Мы группируем train датасет по contract_id и
берем среднее у каждого подрядчика. Далее мы проходимся по каждой строчке из теста и находим его ближайшего
соседа по графу расстояний между подрядчиками. Далее мы пользуемся проведенным нами анализом данным(у одного
контракт айди практически никогда не изменяется вероятность дефолта) и делаем новую строчку, в которой контракт
фичи от нынешнего контракта, а контрактор фичи от самого близкого подрядчика. В итоге мы предсказываем вероятность
дефолта для новой строчки и записываем в финальные предсказания среднее между вероятностями старой и новой строчки.

In [82]:
def check_norm_contract(copy_train, copy_test1, X_simple, dd, f, simple_model):
    data = copy_train.groupby(['contractor_id'], as_index=False).mean()
    scores = []
    for i in tqdm(range(len(copy_test1))):
        if copy_test1.iloc[i, :]['contractor_id'] in dd:
            simple_row = copy_test1[X_simple.columns].iloc[[i], :]
            contractor_mean_data = data[copy_train.contractor_id == dd[copy_test1.iloc[i, :]['contractor_id']][0]]

            row_without_contractor = simple_row.drop(columns=[column for column in simple_row.columns if 'contractor' in column])
            contractor_info = contractor_mean_data.filter(like='contractor')
            if len(contractor_info) == 0:
                scores.append(f.iloc[i][2])
            else:
                row_without_contractor[contractor_info.columns] = contractor_info.iloc[0]
                scores.append((simple_model.predict_proba(row_without_contractor[X_simple.columns])[:, 1][0] + f.iloc[i][2]) / 2)        
        else:
            scores.append(f.iloc[i][2])
    
    return scores

**Cross-validation**

Чтобы протестировать модель на переобучение и понять, как модель работает на других данных, используется перекрестная проверка.
Мы выбрали схему GroupKFold по контракту_id
GroupKFold — вариант увеличения в k раз, обеспечивающий
что одна и та же группа не будет представлена ​​ни в тестовой, ни в обучающей выборке. 
Например, если данные получены от разных субъектов с использованием нескольких образцов для каждого субъекта,
и если модель достаточно гибкая, чтобы учиться на индивидуальных особенностях, 
невозможно будет распространить его на новые темы. GroupKFold позволяет обнаружить такого рода ситуации переобучения.
Каждый предмет проходит разные этапы тестирования, и один и тот же предмет никогда не проходит одновременно и тестирование, и обучение.

In [ ]:
n_splt = 6  # number of the folds
scores = []
recall = []
precision = []

skf = GroupKFold(n_splits=n_splt)
temp = X.copy()

for i, (train_index, val_index) in tqdm(enumerate(skf.split(X, y, ctrs))):
    train_x, test_x = X.iloc[train_index], X.iloc[val_index]
    train_y, test_y = y.iloc[train_index], y.iloc[val_index]

    model = RandomForestClassifier(n_estimators=5000, n_jobs=-1)
    model.fit(train_x[total], train_y)
    preds1 = model.predict_proba(test_x[total])[:, 1]

    model = CatBoostClassifier(eval_metric='AUC', verbose=False)
    model.fit(train_x[total], train_y)
    preds2 = model.predict_proba(test_x[total])[:, 1]

    preds = preds1 * 0.8 + preds2 * 0.2

    scores.append(2 * roc_auc_score(test_y, preds) - 1)
    recall.append(recall_score(test_y, preds.round()))
    precision.append(precision_score(test_y, preds.round()))

In [ ]:
np.mean(scores)  # gini

**Model selection**

Далее сделаем класс модели, так как будем использовать для каждых данных
2 модели - RandomForestClassifier и CatBoostClassifier. После этого
подберем гиперпараметры и обучим модели, сблендим предсказания.

**Подбор гиперпараметров**

In [84]:
# Попробуем использовать библиотеку LazyClassifier для того, чтобы увидеть, какие модели лучше всего подойдут

In [85]:
def gini(y_true, y_pred):
    return 2 * roc_auc_score(y_true, y_pred) - 1

In [ ]:
gini_metric = make_scorer(gini, needs_proba=True)

In [ ]:
cb_params = {
    'iterations': [1000, 2000, 3000, 5000],
    'depth': [6, 8, None],
    "random_strength": [1],
    "verbose": [0],
    "thread_count": [-1]
}

In [ ]:
rf_params = {
    'bootstrap': [True, False],
    'criterion': ['gini'],
    'max_depth': [None, 10, 15],
    'max_features': ['sqrt', None],
    'n_estimators': [3000, 5000],
    'n_jobs': [-1],
    'verbose': [0],
}

In [89]:
cb_params = { # можно не запускать, дальше записаны лучшие параметры
    'iterations': [1000, 2000, 3000],
    'l2_leaf_reg': [1, 2, 3, 5, 7],
    'leaf_estimation_iterations': [5, 7, 10, 12],
    'max_leaves': [60, 61, 62, 63, 64, 65, 70],
    'depth': [4, 6, 10, 15],
    "random_strength": [1],
    "border_count": [250, 251, 253, 254, 255],
    "verbose": [0],
    "thread_count": [-1]
}

In [ ]:
scores_rf = []
recall_rf = []
precision_rf = []

scores_cb = []
recall_cb = []
precision_cb = []

# Разделяем перебор параметров для RandomForest и CatBoost
for rf_param in tqdm(ParameterGrid(rf_params)):
    temp_scores = []
    temp_recall = []
    temp_precision = []

    for i, (train_index, val_index) in enumerate(skf.split(X, y, ctrs)):
        train_x, test_x = X.iloc[train_index], X.iloc[val_index]
        train_y, test_y = y.iloc[train_index], y.iloc[val_index]

        rf_model = RandomForestClassifier(**rf_param)
        rf_model.fit(train_x[total], train_y)
        preds_rf = rf_model.predict_proba(test_x[total])[:, 1]

        temp_scores.append(gini(test_y, preds_rf))
        temp_recall.append(recall_score(test_y, preds_rf.round()))
        temp_precision.append(precision_score(test_y, preds_rf.round()))

    # Вычисляем средние метрики для текущей комбинации параметров RF
    scores_rf.append(np.mean(temp_scores))
    recall_rf.append(np.mean(temp_recall))
    precision_rf.append(np.mean(temp_precision))

for cb_param in tqdm(ParameterGrid(cb_params)):
    temp_scores = []
    temp_recall = []
    temp_precision = []

    for i, (train_index, val_index) in enumerate(skf.split(X, y, ctrs)):
        train_x, test_x = X.iloc[train_index], X.iloc[val_index]
        train_y, test_y = y.iloc[train_index], y.iloc[val_index]

        cb_model = CatBoostClassifier(eval_metric='AUC', **cb_param)
        cb_model.fit(train_x[total], train_y)
        preds_cb = cb_model.predict_proba(test_x[total])[:, 1]

        temp_scores.append(gini(test_y, preds_cb))
        temp_recall.append(recall_score(test_y, preds_cb.round()))
        temp_precision.append(precision_score(test_y, preds_cb.round()))

    scores_cb.append(np.mean(temp_scores))
    recall_cb.append(np.mean(temp_recall))
    precision_cb.append(np.mean(temp_precision))

In [ ]:
best_index_rf = np.argmax(scores_rf)
best_rf_params = list(ParameterGrid(rf_params))[best_index_rf // len(cb_params)]
best_index_cb = np.argmax(scores_cb)
best_cb_params = list(ParameterGrid(cb_params))[best_index_cb % len(cb_params)]

print(f'Best RF params: {best_rf_params}')
print(f'Best CatBoost params: {best_cb_params}')

In [ ]:
best_cb_params

**Model fit**

In [ ]:
class Model:
    def __init__(self):
        """Model class, which is based on 2 classification models: RandomForestClassifier and CatBoostClassifier.
        The class implements the fit, predict_proba and save_model methods.
        """
        self.best_rf_params = {
            'bootstrap': True,
            'criterion': 'gini',
            'max_depth': None,
            'max_features': 'sqrt',
            'n_estimators': 5000,
            'n_jobs': -1,
            'verbose': 0
        }
        self.best_cb_params = {
            'depth': 6,
            'iterations': 1000,
            'random_strength': 1,
            'thread_count': -1,
            'verbose': 0
        }
        self.rf_model = RandomForestClassifier(**self.best_rf_params)
        self.cb_model = CatBoostClassifier(**self.best_cb_params)

    def fit(self, X: pd.DataFrame, y: pd.Series):
        """Fit Classification algorithms to X_train and y_train.
        Parameters
        ----------
        X : array-like,
            Training vectors, where rows is the number of samples
            and columns is the number of features.
        y : array-like,
            Training vectors, where rows is the number of samples
            and columns is the number of features.
        """
        self.rf_model.fit(X, y.map(int))
        self.cb_model.fit(X, y.map(int))

    def predict_proba(self, data: pd.DataFrame, class_=1) -> pd.Series:
        """Prediction probability algorithms to data.
        Parameters
        ----------
        data : array-like,
            Testing vectors, where rows is the number of samples
            and columns is the number of features.
        class_ : int,
            Number of class you need to predict
        Returns
        -------
        predictions : Pandas DataFrame
            Returns predictions of all the models in a Pandas DataFrame.
        """
        self.preds1 = self.rf_model.predict_proba(data)[:, class_]
        self.preds2 = self.cb_model.predict_proba(data)[:, class_]
        return self.preds1 * 0.8 + self.preds2 * 0.2

    def save_model(self, path_rf: str, path_cb: str):
        """Model save algorithms.
        Parameters
        ----------
        path : str,
            String with path you need to save model.
        """
        joblib.dump(self.rf_model, path_rf)
        self.cb_model.save_model(path_cb)
    
    def load_model(self, path_rf: str, path_cb: str):
        """Model load algorithms.
        Parameters
        ----------
        path : str,
            String with path you need to save model.
        """
        self.rf_model = joblib.load(path_rf)
        self.cb_model = CatBoostClassifier()
        self.cb_model.load_model(path_cb)
        return self

In [92]:
model = Model() # proc default6 model
model.fit(X[total], y)

In [93]:
model1 = Model() # top 5 neighbours model
model1.fit(X2[total1], y)

In [94]:
model2 = Model() # baseline model
model2.fit(X3[total2], y)

In [95]:
model.save_model('final/model_rf.joblib', 'final/model_cb.cbm')
model1.save_model('final/model_rf1.joblib', 'final/model_cb1.cbm')
model2.save_model('final/model_rf2.joblib', 'final/model_cb2.cbm')
ts.save_model('final/timeseries.joblib')

**Submission**

Сблендим предсказания лучших моделей в соотношении 5:2:3.
Применим временные ряды и изменим предсказания

In [97]:
copy_test = test.copy()
temp1 = df_imputed[len(train):].reset_index().drop(columns='index')
temp2 = df_imputed1[len(train):].reset_index().drop(columns='index')
temp3 = df_imputed2[len(train):].reset_index().drop(columns='index')
preds1 = model.predict_proba(temp1[total])
preds2 = model1.predict_proba(temp2[total1])
preds3 = model2.predict_proba(temp3[total2])
copy_test = test[['contract_id', 'report_date']]
preds = preds1 * 0.5 + preds2 * 0.2 + preds3 * 0.3 # blending predictions
copy_test['score'] = preds

In [98]:
temp1['score_from_cross_val'] = preds # for stemming
temp = temp1.reset_index()
temp['c'] = range(len(test))
temp['contractor_id'] = contractors2
temp['score'] = preds
s2 = ts.time_series_predict(temp)[0]
score = s2.score

100%|██████████| 595/595 [00:00<00:00, 1021.46it/s]


In [99]:
f = copy_test[['contract_id', 'report_date', 'score']]
f['score'] = score
f

,contract_id,report_date,score
0,3029,2023-07-30,0.03
1,4350,2023-07-30,0.22
2,1095,2023-07-30,0.03
3,2634,2023-07-30,0.14
4,6535,2023-07-30,0.02
...,...,...,...
13211,650,2023-10-29,0.12
13212,4277,2023-10-29,0.13
13213,7316,2023-10-29,0.30
13214,7113,2023-10-29,0.17


In [100]:
scores = check_norm_contract(copy_train, copy_test1, X_simple, dd, f, simple_model)

100%|██████████| 13216/13216 [02:11<00:00, 100.70it/s]


In [ ]:
def explain_predictions(model, data, top=5):
    """
    Функция для объяснения вероятности предсказания модели CatBoost для нескольких строк.
    Параметры:
        - model: обученная модель CatBoostClassifier
        - data: DataFrame с данными для объяснения
    Возвращает:
        DataFrame с текстовыми объяснениями вкладов признаков в вероятность предсказания.
    """
    
    data_pool = Pool(data)
    
    shap_values = model.get_feature_importance(type="ShapValues", data=data_pool)
    
    feature_contributions = shap_values[:, :-1]
    predictions = shap_values[:, -1]
    
    explanations = []
    
    medians = data.median()
    
    for i in tqdm(range(len(data))):
        feature_info = [
            (feature_name, feature_value, feature_contributions[i, j])
            for j, (feature_name, feature_value) in enumerate(zip(data.columns, data.iloc[i]))
        ]
        
        if predictions[i] > 0.5:
            sorted_features = sorted([f for f in feature_info if f[2] > 0], key=lambda x: abs(x[2]), reverse=True)
        else:
            sorted_features = sorted([f for f in feature_info if f[2] < 0], key=lambda x: abs(x[2]), reverse=True)
        
        explanation = []
        
        for i, (feature_name, feature_value, contribution) in enumerate(sorted_features[:top]):
            threshold = medians[feature_name]
            sign = '+' if contribution > 0 else '-'
            explanation.append(f"{i+1}) {features_df[features_df.колонка == feature_name].описание.values[0].strip()} значение {feature_value:.3f} {'>' if feature_value > threshold else '<='} чем медиана {threshold} -> {sign}{abs(contribution):.3f} к вероятности")
        
        explanations.append("\n".join(explanation))
    
    return pd.DataFrame({"interpretation": explanations})

# Пример использования
expl = explain_predictions(model.cb_model, temp1)
expl

In [ ]:
f['interpretation'] = expl.interpretation

In [ ]:
f.score = pd.Series(scores)

In [ ]:
f.to_csv('sussubmit.csv', index=False)

In [ ]:
scores = []
d2 = f.copy()
d2['report_date'] = pd.to_datetime(d2['report_date']).astype('int64') / 10 ** 11

In [ ]:
d2['key'] = list(range(len(d2)))

grouped = d2.groupby('contract_id')

scores = []

for i, row in tqdm(d2.iterrows(), total=len(d2)):
    group = grouped.get_group(row['contract_id'])
    
    group['date_diff'] = (group['report_date'] - row['report_date']).abs()
    
    group_filtered = group[group['key'] != i]
    
    if group_filtered.empty:
        scores.append(row['score'])
    else:
        nearest_rows = group_filtered.nsmallest(5, 'date_diff')
        
        if len(nearest_rows) == 1:
            scores.append(nearest_rows.iloc[0]['score'])
        else:
            avg_score = nearest_rows['score'].mean()
            scores.append(avg_score)


In [ ]:
f['score_nearest'] = scores

In [ ]:
f['score'] = (f['score_nearest']+f['score']*0.4)/1.4

In [ ]:
f.drop(columns=['score_nearest']).to_csv('sussubmit.csv', index=False)

Мы проделали огромную работу. Проанализировали новые данные,
выявили закономерности, поработали с графами и получили score 0.555,
что считаем очень хорошим показателем. Далее планируем тюнить параметры наших алгоритмов и моделей,
что может еще сильнее увеличить показатели скора, помимо этого, также улучшить алгоритм временных рядов.
Мы старались по максимуму комментировать то, что делаем, это должно было сделать код более понятным.
Выполняется он без ошибок. Также провели глубокий анализ данных в ноутбуке EDA.ipynb, проверили гипотезы.
Создали множество доп. признаков, использовали продвинутые методы для обработки графов, подобрали гиперпараметры для моделей.
Большое спасибо, что уделили время и посмотрели наше решение. Хорошего вам дня!